# Recipe Parse

## Define Corpora

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
import requests
import re
from bs4 import BeautifulSoup
import string
import pandas as pd
import numpy as np
import unit_conversion as uc

In [2]:
lemma = nltk.wordnet.WordNetLemmatizer()


def create_ingredient_corpus():
    food1 = wn.synset('food.n.01')
    food2 = wn.synset('food.n.02')
    food3 = wn.synset('food.n.03')
    f1 = list(set([w.lower() for s in food1.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
    f2 = list(set([w.lower() for s in food2.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
    f3 = list(set([w.lower() for s in food3.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
    ings = list(set(f1 + f2 + f3))
    for i in range(len(ings)):
        ings[i] = lemma.lemmatize(ings[i])
        
    return list(set(ings))


ingredient_corpus = create_ingredient_corpus()
len(set(ingredient_corpus))

3465

In [3]:
unit_corpus = list(set(['oz', 'ounces', 'ounce', 'gram', 'grams', 'ml', 'l', 'pound', 'lb',
        'ozs', 'stone', 'st', 's.t.', 'milliliters', 'ton', 't', 'micrograms',
        'microgram', 'kilograms', 'kg', 'kilogram', 'metric ton', 'mt', 'm.t',
        'metric tonne','liter', 'tsp', 'teaspoons', 'teaspoon', 'tbsp',
        'tablespoons', 'cup', 'cups', 'c', 'floz', 'fluid oz', 'fluid ounces', 
        'quart', 'qu', 'qt', 'pint', 'pt', 'gallons', 'gal', 'tablespoon', 'tablespoons',
                       'pinch', 'as needed', 'tiny pinch', 'drops', 'gallon', 'drop', 'dash', 'dashes']))

In [4]:
def add_to_corpus(ingredient, c):
    ingredient = ingredient.split()
    for i in range(len(ingredient)):
        ingredient[i] = lemma.lemmatize(ingredient[i].lower().strip())
    ingredient = '_'.join(ingredient)
    return list(set(c + [ingredient]))


ingredient_corpus = add_to_corpus('canola oil', ingredient_corpus)
ingredient_corpus = add_to_corpus('olive oil', ingredient_corpus)
ingredient_corpus = add_to_corpus('garam masala', ingredient_corpus)
ingredient_corpus = add_to_corpus('vegetable oil', ingredient_corpus)

## Sample Recipe Scrape

In [5]:
def example_recipe_scrape(num):
    """
    Sample code to produce a list of ingredients for a given recipe.
    The example recipe is 'Whole-wheat-pancakes-from-scratch' from the 'Pancakes' subcategory under the 'Breakfast and Brunch' category.
    :return: list of string ingredients. These need to be parsed.
    """
    example_sub_category = 'https://www.allrecipes.com/recipes/' + num + '/breakfast-and-brunch-pancakes/'
    page = requests.get(example_sub_category)
    soup = BeautifulSoup(page.content)
    grid = soup.find('div', attrs={'class': 'fixed-grid'})
    article = grid.find('article', attrs={'class': 'fixed-recipe-card'})
    example_recipe_url = article.find('a').get('href')
    recipe_request = requests.get(example_recipe_url)
    recipe_soup = BeautifulSoup(recipe_request.content)
    ingredients_space = recipe_soup.find('div', attrs={'id': 'polaris-app'})
    ingredients_soup = ingredients_space.find_all('label', attrs={'ng-class': '{true: \'checkList__item\'}[true]'})
    ingredients = []
    for ingredient in range(len(ingredients_soup)):
        ingredients.append(ingredients_soup[ingredient]['title'])

    return ingredients

In [6]:
example_recipe_scrape(str(152))

C:\Users\natha\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\natha\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


['2 1/4 pounds potatoes, scrubbed',
 '3 tablespoons butter',
 '1 teaspoon salt',
 '2 tablespoons milk']

In [7]:
def remove_parens(string):
    while True:
        if '(' in string:
            left = string.index('(')
            right = string.index(')')
            if left > 0:
                if right == len(string) - 1:
                    string = string[:left] # there is nothing to the right of the right parenthesis
                elif right < len(string) - 2:
                    string = string[:left] + string[right+2:] # get rid of underscore after right parenthesis
                elif right < len(string) - 1:
                    string = string[:left] + string[right+1:]
            elif left == 0:
                string = string[right+1:] # there is nothing to the left of the left parenthesis
        elif ')' in string: # only needed when there is a set of parentheses within another
            ind = string.index(')')
            if ind == len(string) - 1:
                string = string[:ind]
            else:
                string = string[:ind] + string[ind+1:]
        else:
            break
        string = string.strip()
    return string

In [8]:
remove_parens('2 (10.75 ounce) cans condensed cream of potato soup')

'2 cans condensed cream of potato soup'

In [9]:
def preprocess_ingredient(string):
    string = string.lower()
    if ',' in string:
        comma = string.index(',')
        # truncate column name to before the comma (assuming directions to the right)
        string = string[0:comma] 
    return string

In [10]:
def determine_unit(recipe):
    # Assume that the parameter, recipe, is a string of the entire recipe but has been fully preprocessed
    # for parentheses and commas
    
    # Loop the elements of the unit corpus through the recipe string
    # add spces to insure that the search does not simply find the instance of a character in the recipe 
    size = len(unit_corpus)
    unit = ''
    for i in range(size):
        un = ' ' + unit_corpus[i] + ' '
        if un in recipe:
            unit = un.strip()
            break
    return(unit)

In [11]:
def find_frac(cur):
    # Assume that the parameter, recipe, is a string of the entire recipe but has been fully preprocessed
    # for parentheses and commas
    
    try:
        first = cur[0]
        second = cur[1]
    except:
        first = cur[0]
        second = ''
        print('Look at this:', first, cur)
    if '/' in first:
        # Then there is a fractional measurement in the first "word" of the recipe
        frac = first.split('/') # split numerator from denominator
        num, denom = (frac[0], frac[1])
        # make sure / isn't just the 'or' symbol
        try:
            num = int(num)
            denom = int(denom)
            measurement = num / denom
            quantity = False
        # If it is, default the measurement to 1 'quantity'
        except: 
            measurement = 1
            quantity = True
        cur = cur[1:]
        ingred = '_'.join(cur)
    elif '/' in second: # the second word of the recipe is a fraction (e.g. 1 3/4 cups)
            frac = second.split('/')
            try:
                num, denom = (frac[0], frac[1])
                num = int(num)
                denom = int(denom)
                measurement = int(first) + num / denom
                quantity = False
            except:
                measurement = 1
                quantity = True
            cur = cur[2:]
            ingred = '_'.join(cur)
    else:
        # Check if the first word of the ingredient is even a number at all
        try:
            measurement = int(first)
            ingred = '_'.join(cur[1:])
            quantity = False
        except:
            # In this case, there is not a measurement for this ingredient
            # It is itself a unit, e.g. cooking spray
            measurement = 1
            quantity = True
            ingred = '_'.join(cur)
    return(ingred, measurement, quantity)

In [12]:
def parse_recipe(recipe, dictionary={}, col_list=[], measurement_list=[], iteration=0):
    """
    Extract the measurement and unit of the ingredient as well as whether or not it should be expressed as a quantity
    An ingredient, for our purposes, is expressed as EITHER a measurement (e.g. 3 referring to cups) OR a quantity 
    (e.g., 3 referring to eggs)
    Quantity will be set to True or False based on these observations
    :param recipe: an array where each entry is as follows: (measurement, unit, ingredient)
    """
    ings = len(recipe)
    for i in range(ings):
        # remove parentheses from ingredients
        recipe[i] = remove_parens(recipe[i])
        recipe[i] = preprocess_ingredient(recipe[i])
        cur = recipe[i].split() # make the ingredient into an array itself!
        # lemmatize the ingredient specification
        for j in range(len(cur)):
            cur[j] = lemma.lemmatize(cur[j])
#             if ',' in cur[j]:
#                 comma = cur[j].index(',')
#                 word = cur[j][:comma]
        recipe[i] = ' '.join(cur)
        ind = 0
        # If the first word is 'about', check the next word for the measurement
        if cur[ind].lower().strip() == 'about': 
            ind += 1
           
#        try:
#             unit = determine_unit(recipe[i])
#             recipe[i].replace(unit, '') # remove unit from recipe string
#             first = cur[ind]
#             second = cur[ind+1]
#             ingred = '_'.join(cur[ind+2:]) # the rest of the entry is the ingredient specification joint by underscore
#         except: # no second with a fraction in it
#             recipe[i].replace(unit, '')
#             first = cur[ind]
#             second = ''
#             ingred = '_'.join(cur[ind+2]:)

        unit = determine_unit(recipe[i])
        if unit != '':
            recipe[i] = recipe[i].replace(unit + ' ', '') # remove unit from recipe string
        cur = recipe[i].split()
        
        ingred, measurement, quantity = find_frac(cur)
                        
        # If quantity is False, we do a unit conversion of the found measurement. However, if the found "unit"
        # (second or third word) does not match a unit in our dictionary, we express the ingredient as a quantty
        # This is checked in the convert function itself
        # We won't worry about making a dataframe just yet. We will first list all the columns and the corresponding values
        # unit = re.sub(r'[^\w\s]','', unit)
        if quantity or unit == '':
            print('quantity')
            print('No conversion necessary')
            col_list.append(ingred)
            measurement_list.append(measurement)
        elif not quantity and unit != '':
            print(unit)
            mes, cols, quantity = uc.convert(measurement, unit, ingred)
            if quantity:
                col_list.append(ingred)
                measurement_list.append(measurement)
                # If we know for sure the unit we found is in our dictionary, we have a valid measurement
            elif not quantity: 
                col_list.extend(cols)
                measurement_list.extend(mes)
    # lemmatize
    for i in range(len(col_list)):
        temp = col_list[i].split('_')
        for j in range(len(temp)):
            temp[j] = lemma.lemmatize(temp[j])
        col_list[i] = '_'.join(temp)
    
    # Now populate the dictionary
    # Either add a value to the list of values corresponding to a key 
    # Or create a new key value pair
    # init key value pairs as col_list and empty lists per column
    if len(dictionary) == 0:
        dictionary = dict(zip(col_list, [[] for i in range(len(col_list))]))
    for i in set(col_list + list(dictionary.keys())):
        if i in col_list:
            if i in dictionary:
                dictionary[i] += [measurement_list[col_list.index(i)]]
            else:
                dictionary[i] = [0]*iteration + [measurement_list[col_list.index(i)]]
        else:
            if i in dictionary:
                dictionary[i] += [0]
            else:
                dictionary[i] = [0]
    # Will be used to construct dataframe
    return(dictionary)

In [13]:
d = parse_recipe(['1/2 cup wheat germ',
 '2 cups whole wheat flour',
 '1 teaspoon baking soda',
 '1/2 teaspoon salt',
 '3 cups buttermilk',
 '2 eggs, lightly beaten',
 '1 tablespoon canola oil',
 'cooking spray'], dictionary = {})
print('\n', d)

cup
Converted cup to milliliters
cup
Converted cup to milliliters
teaspoon
Converted teaspoon to milliliters
teaspoon
Converted teaspoon to milliliters
cup
Converted cup to milliliters
quantity
No conversion necessary
tablespoon
Converted tablespoon to milliliters
quantity
No conversion necessary

 {'wheat_germ_mass': [0], 'wheat_germ_volume': [120.0], 'whole_wheat_flour_mass': [0], 'whole_wheat_flour_volume': [480], 'baking_soda_mass': [0], 'baking_soda_volume': [4.929], 'salt_mass': [0], 'salt_volume': [2.464], 'buttermilk_mass': [0], 'buttermilk_volume': [720], 'egg': [2], 'canola_oil_mass': [0], 'canola_oil_volume': [14.787], 'cooking_spray': [1]}


### Load sample csv 

In [14]:
path = "C:\\Users\\natha\\Desktop\\AdvTopsML\\Recipe-Recommendation-System\\"
path

'C:\\Users\\natha\\Desktop\\AdvTopsML\\Recipe-Recommendation-System\\'

In [15]:
data = pd.read_csv(path + 'data4.csv')
data['ingredients'].head()

0    ['1 cup orange juice', '1 1/2 tablespoons arro...
1    ['1 cup white sugar', '5 tablespoons butter, c...
2    ['1 pint fresh raspberries', '1/4 cup white su...
3    ['4 (1 ounce) squares unsweetened chocolate', ...
4    ['1 pint fresh strawberries', '1/3 cup white s...
Name: ingredients, dtype: object

In [16]:
import ast
ingz = []
for i in range(len(data)):
    ingz.append(ast.literal_eval(data['ingredients'][i]))

In [17]:
size = len(ingz)
print('Number of sauces:', size)
ingz[1]

Number of sauces: 439


['1 cup white sugar',
 '5 tablespoons butter, cut into slices',
 '1/2 cup heavy cream',
 '1 tablespoon heavy whipping cream',
 '1 pinch sea salt to taste']

In [18]:
d = {}
for j in range(size):
    rec = ingz[j]
    print(rec)
    d = parse_recipe(recipe=rec, dictionary=d, col_list=[], measurement_list=[], iteration=j)
    print('\n')

['1 cup orange juice', '1 1/2 tablespoons arrowroot powder', '2 tablespoons lemon juice', '1/3 cup honey', '2 teaspoons lemon zest']
cup
Converted cup to milliliters
tablespoon
Converted tablespoon to milliliters
tablespoon
Converted tablespoon to milliliters
cup
Converted cup to milliliters
teaspoon
Converted teaspoon to milliliters


['1 cup white sugar', '5 tablespoons butter, cut into slices', '1/2 cup heavy cream', '1 tablespoon heavy whipping cream', '1 pinch sea salt to taste']
cup
Converted cup to milliliters
tablespoon
Converted tablespoon to milliliters
cup
Converted cup to milliliters
tablespoon
Converted tablespoon to milliliters
pinch
Converted teaspoon to milliliters


['1 pint fresh raspberries', '1/4 cup white sugar', '2 tablespoons orange juice', '2 tablespoons cornstarch', '1 cup cold water']
pint
Converted pint to milliliters
cup
Converted cup to milliliters
tablespoon
Converted tablespoon to milliliters
tablespoon
Converted tablespoon to milliliters
cup
Converted cu

No conversion necessary
quantity
No conversion necessary
tablespoon
Converted tablespoon to milliliters


['4 cups fresh blackberries', '1 cup sugar', '1 tablespoon brandy', '2 tablespoons cornstarch', '1/4 cup water']
cup
Converted cup to milliliters
cup
Converted cup to milliliters
tablespoon
Converted tablespoon to milliliters
tablespoon
Converted tablespoon to milliliters
cup
Converted cup to milliliters


['1 gallon milk', '1 vanilla bean', '4 1/2 cups white sugar', '1 teaspoon baking soda']
gallon
Converted gallon to milliliters
quantity
No conversion necessary
cup
Converted cup to milliliters
teaspoon
Converted teaspoon to milliliters


['2 cups semisweet chocolate chips', '1 cup butter', "4 cups confectioners' sugar", '2 2/3 cups evaporated milk', '1 teaspoon vanilla extract', '1/8 teaspoon salt']
cup
Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters
teaspoon
Converted teaspoon to milliliters
teaspoon

Converted teaspoon to milliliters
teaspoon
Converted teaspoon to milliliters


['1 cup white sugar', '1/2 cup butter', '1/2 cup light corn syrup', '1/2 cup frozen orange juice concentrate, thawed']
cup
Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters


['1 cup white sugar', '1/2 cup water', '3/4 cup heavy whipping cream', '3 tablespoons unsalted butter', '1 teaspoon sea salt (such as Maldon(R))']
cup
Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters
tablespoon
Converted tablespoon to milliliters
teaspoon
Converted teaspoon to milliliters


['2 cups diced rhubarb', '2 cups white sugar']
cup
Converted cup to milliliters
cup
Converted cup to milliliters


['1 1/2 cups brown sugar', '3/4 cup unsweetened cocoa', '1/4 cup all-purpose flour', '1 cup water', '1 (12 fluid ounce) can evaporated milk', '2 tablespoons butter', '2 teaspoons vanilla extract']
cup
Converted cup 

Converted tablespoon to milliliters
teaspoon
Converted teaspoon to milliliters
tablespoon
Converted tablespoon to milliliters


['1 cup heavy cream', '1 tablespoon plain yogurt']
cup
Converted cup to milliliters
tablespoon
Converted tablespoon to milliliters


['1 (14 ounce) can sweetened condensed milk (such as Eagle Brand(R))', '1 (12 ounce) jar caramel topping', '6 ounces semisweet chocolate chips', '1/4 cup coffee-flavored liqueur (such as Kahlua(R)), or more to taste']
quantity
No conversion necessary
quantity
No conversion necessary
ounce
Converted teaspoon to milliliters
cup
Converted cup to milliliters


['4 Cortland apples - peeled, cored, and chopped', '2 Golden Delicious apples - peeled, cored, and chopped', '2 Bartlett pears - peeled, cored, and chopped', '1/4 cup water', '1/2 teaspoon ground cinnamon', '1 pinch ground nutmeg (optional)', '2 tablespoons maple syrup']
quantity
No conversion necessary
quantity
No conversion necessary
quantity
No conversion necessary
cup
Conve

cup
Converted cup to milliliters
cup
Converted cup to milliliters
tablespoon
Converted tablespoon to milliliters
teaspoon
Converted teaspoon to milliliters
cup
Converted cup to milliliters


['2 cups heavy whipping cream', '1/2 cup butter', '1 1/2 cups brown sugar', '1/2 cup white sugar', '1/2 teaspoon ground cinnamon', '1/4 teaspoon grated nutmeg', '1/8 teaspoon salt']
cup
Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters
teaspoon
Converted teaspoon to milliliters
teaspoon
Converted teaspoon to milliliters
teaspoon
Converted teaspoon to milliliters


["1/2 (10 ounce) jar Dickinson's(R) Lemon Curd", "1/2 (10 ounce) jar Dickinson's(R) Pure Marion Blackberry Preserves or any berry flavor Dickinson's(R) Preserves or Purely Fruit(R) Spreadable Fruit", '1 (8 ounce) package cream cheese, softened']
quantity
No conversion necessary
quantity
No conversion necessary
quantity
No conversion necessary


['1 (10 ounce) pa

...

tablespoon
Converted tablespoon to milliliters


['1 1/2 cups crushed ice', '1 ripe banana', '1/2 lime, juiced', '1/2 lemon, juiced', '1 tablespoon shredded coconut, as garnish (optional)', '2 pitted cherries, as garnish (optional)']
cup
Converted cup to milliliters
quantity
No conversion necessary
quantity
No conversion necessary
quantity
No conversion necessary
tablespoon
Converted tablespoon to milliliters
quantity
No conversion necessary


['4 pounds rhubarb, cut into 1-inch pieces', '2 cups white sugar', '1 1/3 cups raisins', '1/4 cup vegetable shortening', '2/3 cup white sugar', '2 eggs', '1 teaspoon vanilla extract', '2 cups all-purpose flour', '1 tablespoon baking powder', '1/2 teaspoon salt', '2/3 cup milk']
pound
Converted teaspoon to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters
quantity
No conversion necessary
teaspoon
Converted teaspoon to milliliters
cup
Converted cup to mill

Converted teaspoon to milliliters


['3/4 cup vanilla sugar-free yogurt', '1/2 cup frozen blueberries', '8 frozen strawberries']
cup
Converted cup to milliliters
cup
Converted cup to milliliters
quantity
No conversion necessary


['1 1/2 cups butter, softened', '1 cup granular sucralose sweetener (such as Splenda(R))', '1/3 cup white sugar', '4 eggs', '1 teaspoon vanilla extract', '5 cups all-purpose flour', '2 teaspoons baking powder']
cup
Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters
quantity
No conversion necessary
teaspoon
Converted teaspoon to milliliters
cup
Converted cup to milliliters
teaspoon
Converted teaspoon to milliliters


['2 cups sliced strawberries', '1/4 cup balsamic vinegar', '1 tablespoon white sugar', '1 pinch ground black pepper', '4 slices angel food cake', '1/2 cup whipped cream']
cup
Converted cup to milliliters
cup
Converted cup to milliliters
tablespoon
Converted tablespoon to milliliters
pinch
Converted teaspo

Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters
quantity
No conversion necessary


['9 medium oranges', '2 1/2 cups orange juice', '2 tablespoons honey', '4 teaspoons unflavored gelatin', '3 tablespoons chopped fresh mint', '1 tablespoon chopped fresh mint']
quantity
No conversion necessary
cup
Converted cup to milliliters
tablespoon
Converted tablespoon to milliliters
teaspoon
Converted teaspoon to milliliters
tablespoon
Converted tablespoon to milliliters
tablespoon
Converted tablespoon to milliliters


['1 cup cubed cantaloupe', '1/2 cup cubed honeydew', '2 cups cubed watermelon', '1/2 cup diced apple', '1 cup sliced banana', '1 cup cubed fresh mango', '3 cups lemon-lime flavored carbonated beverage, chilled']
cup
Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters

Converted cup to milliliters
cup
Converted cup to milliliters


['1 1/4 cups white sugar', '1/2 cup water', '1/4 cup pear liqueur', '1 pound fresh strawberries, hulled', '1 pint blueberries', '1 pint fresh blackberries']
cup
Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters
pound
Converted teaspoon to milliliters
pint
Converted pint to milliliters
pint
Converted pint to milliliters


['1 (3 ounce) package strawberry Jell-O(R) mix', '1 cup boiling water', '1/2 cup cold water', '3 empty 6-ounce juice cans', '3 banana halves, cut crosswise', '3 (4 inch) pieces red string licorice candy']
quantity
No conversion necessary
cup
Converted cup to milliliters
cup
Converted cup to milliliters
quantity
No conversion necessary
quantity
No conversion necessary
quantity
No conversion necessary


['4 fresh plums, pitted and halved', '2 fresh nectarines, pitted and halved', '2 tablespoons honey']
quantity
No conversion necessary
quantity
No conversion necess

cup
Converted cup to milliliters
cup
Converted cup to milliliters
quantity
No conversion necessary
teaspoon
Converted teaspoon to milliliters
quantity
No conversion necessary
cup
Converted cup to milliliters


['1 grapefruit, cut in half', '1/4 teaspoon ground cinnamon', '1/4 teaspoon ground nutmeg', '1/2 teaspoon coconut sugar, or more to taste']
quantity
No conversion necessary
teaspoon
Converted teaspoon to milliliters
teaspoon
Converted teaspoon to milliliters
teaspoon
Converted teaspoon to milliliters


['2 cups blueberry jam', '1 1/2 cups nonfat Greek yogurt', '1/2 cup almond milk', '3 tablespoons stevia sweetener (such as Truvia(R)), or more to taste', '1 tablespoon lemon juice', '1 teaspoon vanilla extract', '1/4 teaspoon sea salt', '1/8 teaspoon ground cinnamon']
cup
Converted cup to milliliters
cup
Converted cup to milliliters
cup
Converted cup to milliliters
tablespoon
Converted tablespoon to milliliters
tablespoon
Converted tablespoon to milliliters
teaspoon
Converted teasp

In [19]:
df = pd.DataFrame(data = np.array(list(d.values())).T, columns = np.array(list(d.keys())))

In [ ]:
df.head(10)

,orange_juice_mass,orange_juice_volume,arrowroot_powder_mass,arrowroot_powder_volume,lemon_juice_mass,lemon_juice_volume,honey_mass,honey_volume,lemon_zest_mass,lemon_zest_volume,...,blueberry_jam_volume,blueberry_jam_mass,dark_chocolate_chip_volume,salted_peanut_volume,dark_chocolate_chip_mass,salted_peanut_mass,instant_espresso_powder_mass,instant_espresso_powder_volume,high-quality_unsweetened_cocoa_powder_volume,high-quality_unsweetened_cocoa_powder_mass
0,0.0,240.000,0.0,22.18,0.0,29.574,0.0,80.0,0.0,9.858,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,29.574,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,240.000,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
